In [5]:
#env: equake_env (python 3.9)
#import dependecies
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')

import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func


In [7]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [8]:
#create and connect to a sqlite3 db
path = "../Database/db/earthquake_db"
conn = sqlite3.connect(path)

In [9]:
#for sql database read the csv with the raw data or obtain it from the postgresql database or sqllite db

# raw_data = pd.read_csv(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/csv/earthquake_raw.csv')
# raw_data

raw_data = pd.read_sql_query('select * from "earthquake_raw"',con=engine)
raw_data

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.1,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.0,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.2,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.8,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.6,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58106,2022-05-10T18:52:32.159Z,-27.668500,-176.934500,25.08,4.60,mb,NaN,76.00,1.80900,0.7400,...,2022-05-31T22:12:12.040Z,Kermadec Islands region,earthquake,10.1,3.7,0.100,34.0,reviewed,us,us
58107,2022-05-10T18:49:23.310Z,-59.013100,-25.575000,55.95,4.90,mb,NaN,65.00,7.65300,0.6000,...,2022-05-31T19:19:19.040Z,South Sandwich Islands region,earthquake,11.6,4.4,0.099,32.0,reviewed,us,us
58108,2022-05-10T16:40:16.050Z,34.731833,-119.073333,11.19,0.83,ml,8.0,168.00,0.02522,0.1500,...,2022-06-07T17:55:46.520Z,"14km SSE of Pine Mountain Club, CA",earthquake,0.5,0.9,0.211,2.0,reviewed,ci,ci
58109,2022-05-10T08:28:40.129Z,2.945500,128.323100,53.98,4.50,mb,NaN,100.00,2.36100,0.8200,...,2022-06-06T07:24:34.040Z,"139 km NNE of Tobelo, Indonesia",earthquake,7.0,7.8,0.111,24.0,reviewed,us,us


In [9]:
#obtain data from the USGS website to update/add more rows of information
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-25T02:30:26.923Z,35.847630,-98.084999,4.339945,2.08,ml,41.0,39.444824,0.13235,0.362085,...,2022-01-25T02:33:42.087Z,"13 km W of Kingfisher, Oklahoma",earthquake,1.796045,1.656136,NaN,31.0,automatic,ok,ok
1,2022-01-25T02:25:56.510Z,37.506000,-118.840332,7.410000,1.87,md,18.0,121.000000,0.08435,0.020000,...,2022-01-25T02:34:11.157Z,"15km WSW of Toms Place, CA",earthquake,0.370000,1.030000,0.190,22.0,automatic,nc,nc
2,2022-01-25T02:19:53.472Z,38.166200,-117.822300,7.200000,2.00,ml,24.0,89.330000,0.11100,0.364500,...,2022-01-25T02:37:34.750Z,"35 km SE of Mina, Nevada",earthquake,NaN,0.400000,0.230,12.0,automatic,nn,nn
3,2022-01-25T02:14:32.205Z,9.966100,-59.922900,10.000000,5.30,mww,NaN,66.000000,2.74200,0.740000,...,2022-01-25T02:30:17.040Z,"142 km ESE of Rio Claro, Trinidad and Tobago",earthquake,9.600000,1.900000,0.093,11.0,reviewed,us,us
4,2022-01-25T02:05:32.400Z,33.376333,-116.866833,5.980000,0.84,ml,29.0,38.000000,0.02293,0.210000,...,2022-01-25T02:09:08.848Z,"2km N of Palomar Observatory, CA",earthquake,0.310000,0.670000,0.123,23.0,automatic,ci,ci
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9134,2021-12-26T03:49:14.657Z,-18.838900,-175.303600,192.060000,4.30,mb,NaN,145.000000,5.95400,0.850000,...,2022-01-06T23:33:57.040Z,"140 km W of Neiafu, Tonga",earthquake,18.000000,11.000000,0.153,13.0,reviewed,us,us
9135,2021-12-26T03:20:59.642Z,64.697300,-149.720600,16.400000,1.20,ml,NaN,NaN,NaN,0.690000,...,2022-01-19T17:20:24.380Z,"30 km WNW of Four Mile Road, Alaska",earthquake,NaN,0.200000,NaN,NaN,reviewed,ak,ak
9136,2021-12-26T03:17:12.060Z,38.818833,-122.765167,1.930000,1.04,md,33.0,49.000000,0.01220,0.040000,...,2021-12-31T00:53:11.527Z,"4km W of Cobb, CA",earthquake,0.160000,0.240000,0.221,7.0,reviewed,nc,nc
9137,2021-12-26T03:09:04.430Z,34.196667,-119.599333,0.640000,1.80,ml,18.0,211.000000,0.21360,0.230000,...,2022-01-05T18:04:24.882Z,"20km NNW of Santa Cruz Is. (E end), CA",earthquake,0.580000,0.600000,0.156,7.0,reviewed,ci,ci


In [ ]:
# def sql_update(fname, table_name):
#     df_update = pd.read_csv(fname, index_col=None)
#     df_update.to_sql(table_name, engine, if_exists='replace', index = False)
#     return df_update

# df_update.to_sql('earthquake_model_xgblatlongdepth_depth', engine, if_exists='replace', index = False)

In [10]:
#copy data from dataframe to sqlite db to create it
raw_data.to_sql('earthquake_raw', conn, if_exists='replace', index=False)

# Create a cursor object
cur = conn.cursor()
# Fetch and display result
for row in cur.execute('SELECT * FROM earthquake_raw'):
    print(row)


('2022-01-06T01:42:39.855Z', 37.384, -117.1075, 1.1, 1.3, 'ml', 8.0, 149.46, 0.875, 0.2175, 'nn', 'nn00831163', '2022-01-06T01:47:19.317Z', '37 km SSE of Goldfield, Nevada', 'earthquake', None, 39.1, 0.29, 4.0, 'automatic', 'nn', 'nn')
('2022-01-06T01:31:33.698Z', 38.181, -117.818, 7.3, 1.4, 'ml', 11.0, 154.64, 0.119, 0.1017, 'nn', 'nn00831161', '2022-01-06T01:47:40.245Z', '34 km SE of Mina, Nevada', 'earthquake', None, 2.0, 0.46, 5.0, 'automatic', 'nn', 'nn')
('2022-01-06T01:24:20.083Z', 61.3407, -149.8852, 30.0, 1.5, 'ml', None, None, None, 0.75, 'ak', 'ak0229xypaw', '2022-01-06T01:29:22.015Z', '5 km ESE of Point MacKenzie, Alaska', 'earthquake', None, 0.2, None, None, 'automatic', 'ak', 'ak')
('2022-01-06T01:15:15.851Z', 62.8703, -150.7625, 111.3, 2.1, 'ml', None, None, None, 0.32, 'ak', 'ak0229xwriz', '2022-01-06T01:29:21.875Z', '41 km N of Petersville, Alaska', 'earthquake', None, 0.8, None, None, 'automatic', 'ak', 'ak')
('2022-01-06T01:10:44.629Z', 64.0145, -148.7657, 1.9, 1.5, 

In [14]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

In [10]:
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.1,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.0,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.2,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.8,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.6,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58106,2022-05-10T18:52:32.159Z,-27.668500,-176.934500,25.08,4.60,mb,NaN,76.00,1.80900,0.7400,...,2022-05-31T22:12:12.040Z,Kermadec Islands region,earthquake,10.1,3.7,0.100,34.0,reviewed,us,us
58107,2022-05-10T18:49:23.310Z,-59.013100,-25.575000,55.95,4.90,mb,NaN,65.00,7.65300,0.6000,...,2022-05-31T19:19:19.040Z,South Sandwich Islands region,earthquake,11.6,4.4,0.099,32.0,reviewed,us,us
58108,2022-05-10T16:40:16.050Z,34.731833,-119.073333,11.19,0.83,ml,8.0,168.00,0.02522,0.1500,...,2022-06-07T17:55:46.520Z,"14km SSE of Pine Mountain Club, CA",earthquake,0.5,0.9,0.211,2.0,reviewed,ci,ci
58109,2022-05-10T08:28:40.129Z,2.945500,128.323100,53.98,4.50,mb,NaN,100.00,2.36100,0.8200,...,2022-06-06T07:24:34.040Z,"139 km NNE of Tobelo, Indonesia",earthquake,7.0,7.8,0.111,24.0,reviewed,us,us


In [11]:
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.1,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.0,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.2,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.8,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.6,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58103,2022-05-10T18:52:32.159Z,-27.668500,-176.934500,25.08,4.60,mb,NaN,76.00,1.80900,0.7400,...,2022-05-31T22:12:12.040Z,Kermadec Islands region,earthquake,10.1,3.7,0.100,34.0,reviewed,us,us
58104,2022-05-10T18:49:23.310Z,-59.013100,-25.575000,55.95,4.90,mb,NaN,65.00,7.65300,0.6000,...,2022-05-31T19:19:19.040Z,South Sandwich Islands region,earthquake,11.6,4.4,0.099,32.0,reviewed,us,us
58105,2022-05-10T16:40:16.050Z,34.731833,-119.073333,11.19,0.83,ml,8.0,168.00,0.02522,0.1500,...,2022-06-07T17:55:46.520Z,"14km SSE of Pine Mountain Club, CA",earthquake,0.5,0.9,0.211,2.0,reviewed,ci,ci
58106,2022-05-10T08:28:40.129Z,2.945500,128.323100,53.98,4.50,mb,NaN,100.00,2.36100,0.8200,...,2022-06-06T07:24:34.040Z,"139 km NNE of Tobelo, Indonesia",earthquake,7.0,7.8,0.111,24.0,reviewed,us,us


In [12]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

False


In [13]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

False


In [14]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
58106  2022-05-10T18:52:32.159Z -27.668500 -176.934500   25.08  4.60      mb   
58107  2022-05-10T18:49:23.310Z -59.013100  -25.575000   55.95  4.90      mb   
58108  2022-05-10T16:40:16.050Z  34.731833 -119.073333   11.19  0.83      ml   
58109  2022-05-10T08:28:40.129Z   2.945500  128.323100   53.98  4.50      mb   
58110  2022-05-10T00:19:13.902Z  -4.962200  102.541400   41.84  4.30      mb   

        nst     gap     dmin     rms  .

In [15]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
58103  2022-05-10T18:52:32.159Z -27.668500 -176.934500   25.08  4.60      mb   
58104  2022-05-10T18:49:23.310Z -59.013100  -25.575000   55.95  4.90      mb   
58105  2022-05-10T16:40:16.050Z  34.731833 -119.073333   11.19  0.83      ml   
58106  2022-05-10T08:28:40.129Z   2.945500  128.323100   53.98  4.50      mb   
58107  2022-05-10T00:19:13.902Z  -4.962200  102.541400   41.84  4.30      mb   

        nst     gap     dmin     rms  .

In [24]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

In [25]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

In [28]:
#copy data as csv as backup method
model_df.to_csv("earthquake_raw.csv",index=False)

In [16]:
# Close connection to SQLite database
conn.close()
connection.close()